# **Tensorflow를 이용한 MNIST Multi Layer Perceptron with Early Stopping & Drop out 실습하기**

In [1]:
import tensorflow.compat.v1 as tf
import numpy as np

tf.__version__

'2.4.1'

## **MLP Architecture**
input layer, hidden layer h1, h2, output layer logit. 10개의 output 노드가 10개의 softmax를 통해 prediction으로 변하고, 이 prediction을 y label과 비교를 함으로서 cross entropy를 가짐. 이 cross entropy를 최저로 가지는 것을 목적 함수로 해 adam optimizer를 돌려 architecture를 최적화 한다.
<p align="center"><img src="https://camo.githubusercontent.com/993b7481ad0a8441d1e3393fc6fc493358be08fc/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6d696e73756b2d68656f2f646565706c6561726e696e672f6d61737465722f696d672f64726f706f75742e706e67" height="450"></p>

## **MNIST Data**

In [2]:
# mnist data load
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


train data는 **60000**개의 sample을 가진다.  
test data는 **60000**개의 sample을 가진다.  
모든 data는 **28*28** pixels
  
  MNIST는 사람이 손으로 쓴 숫자로 이루어지는 대형 데이터베이스다. 아래 이미지는 28*28 pexels로 구성되며 흰색과 검정색으로 이루어진 gray scale image [0 to 255]다.

<p align="center"><img src="https://mblogthumb-phinf.pstatic.net/MjAxOTAxMTZfMTg4/MDAxNTQ3NjQ2MzI5OTQ0.257wNc_ErINd8lMDTaor5ls3UP0T2lD1yV_KoC_Ik6Mg.OGV-dRDH3m9U48AhlEaptGI9D8tHsvP-OZbPYyofin4g.PNG.garden2040/Figure_1.png?type=w800" height="300"></p>


## **Split Train data into Train and Validation data**
training 중 validation은 다음과 같은 장점을 가진다.  
1. training 중 validation score를 이용해 hyper parameter를 수정할 수 있다.
2. training score가 올라가는 동안 validation score(accuracy)가 변화가 없을 시 **early stopping**을 제공한다. (**overfitting** 방지)


train data 60000개 중 10000개의 데이터를 validation data로 사용한다.

In [4]:
x_val = x_train[50000:60000]
x_train = x_train[0:50000]
y_val = y_train[50000:60000]
y_train = y_train[0:50000]

## **Reshape**

모든 pixels을 hidden layer에 fully connected (완전 연결)하려면 (28,28)을 (28x28, 1)로 reshape 해야한다.  
== row * col shape를 28x28개의 item이 있는 행렬로 flatten 해야 한다.

<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png" height="420"></p>

In [5]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)


In [6]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

## **Normalize data**
정규화는 모든 입력 feature를 동일한 범위로 만들어 분산을 줄임으로써 학습 속도를 향상시키거나, 더 나은 성능을 돕는다.  
MNIST 데이터 셋은 [0 to 255]의 값을 가지므로 정규화를 할 시 variance만 줄일 수 있다.  
  
MNIST는 MLP architecture로 표준화하는 것보다 정규화하는 것이 더 낫다. ReLU가 peed forward와 back propagation에서 모두 0을 다르게 다루기 때문. MNIST에서 [1 to 255]는 손으로 쓴 것을 의미하므로 아무 것도 쓰지 않은 0을 다르게 취급하는 것이 중요하다.


In [7]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

# 255로 나누게 되면 [0 to 1]의 값을 가지게 됨
gray_scale = 255 
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

## **Label to One Hot Encoding Value**


In [8]:
# class를 10으로 해 y값들을 모두 One Hot Encoding한다. (1~10까지의 숫자를 0~9번 노드로)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [9]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

## **Tensorflow MLP Graph**

<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png" height="450"></p>

## **Drop Out**
Drop out은 hidden layer에서 일부 node를 비활성화하는 것으로(model의 variance를 낮추는 효과가 있다), overfitting을 예방한다.  model을 train 할 때만 사용한다.
Drop out은 모든 mini batch에서 서로 다른 활성화된 node가 있기 때문에 ensemble 효과가 있다.  
Drop out mini batch의 결과로 딥러닝 모델에서 변수를 조정하는 것은 random forest 접근법과 유사하다.  
keep_prob는 drop out 비율에 대한 tensorflow placehoder다.

<p align="center"><img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/08/eba93f5a75070f0fbb9d86bec8a009e9.png" height="350"></p>

In [18]:
# tensorflow 2.x에서는 session을 정의하고 run을 수행하는 과정이 생략되고 바로 실행되는 형태가 되었으므로 아래와 같은 옵션을 추가해준다.
tf.compat.v1.disable_eager_execution()

# input으로 784개
x = tf.compat.v1.placeholder(tf.float32, [None, 784])

# [None, 10] : 총 몇 개의 sample이 들어올지 모르겠지만 10개의 숫자를 구분할 것이다.
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

keep_prob = tf.compat.v1.placeholder(tf.float32)

hidden layer 2에서 drop out을 사용한다.  
keep_prob는 train이나 test에서 채워진다.

In [20]:
def mlp(x):
  # hidden layer1
  w1 = tf.Variable(tf.compat.v1.random_uniform([784, 256])) # 784개의 input을 받고, 256개의 node를 만든다.
  b1 = tf.Variable(tf.zeros([256]))
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1) # activation func : ReLU
  # hidden layer2
  w2 = tf.Variable(tf.compat.v1.random_uniform([256, 128])) # 256개의 input을 받고, 128개의 node를 만든다.
  b2 = tf.Variable(tf.zeros([128]))
  h2 = tf.nn.relu(tf.matmul(h1, w2) + b2) # activation func : ReLU
  h2_drop = tf.nn.dropout(h2, keep_prob) # drop out
  # output layer
  w3 = tf.Variable(tf.compat.v1.random_uniform([128, 10])) # 128개의 input을 받고, 10개의 node를 만든다.
  b3 = tf.Variable(tf.zeros([10]))
  logits = tf.matmul(h2, w3) + b3

  return logits

cross entropy를 하기 위해 softmax를 많이 사용한다. 이 logic value를 softmax에 넣어서 prediction을 받는다. 이 cross entropy를 minimize하는 것이 adam optimizer.

In [21]:
logits = mlp(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

In [23]:
train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

## **Early Stopping**
학습 회차별로, 학습의 정확도는 계속 올라가더라도 검증의 손실은 overfitting으로 오히려 낮아지게 되는 지점이 있는데 이를 과적합으로 간주하여 학습을 종료하는 것을 Early Stopping이라고 한다.
<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/early_stop.png" height="350"></p>



In [25]:
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 300
batch_size = 1000 # 1000개씩 넣으면서 architecture 최적화
iteration = len(x_train) // batch_size # input이 50000개니까 총 50번의 iteration

earlystop_threshold = 5 # validation set accuracy가 내려갔다가 올라가는 경우가 있는데 이를 5번까지 기다려주겠다는 의미
earlystop_cnt = 0

데이터를 제공할 때마다 drop out 비율에 대해 반드시 keep_prob를 채워야 한다.  
hidden layer 2에서 node 10%를 삭제하기 위해 0.9를 train에서 사용한다.  
test중에 dropout을 사용하지 않을 것이므로 test에서 1.0을 사용했다.

In [27]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], feed_dict={x: x_train[start: end], y: y_train[start: end], keep_prob: 0.9}) # training 할 때 90%만 사용한다(keep_prob)
            start += batch_size; end += batch_size
            # Compute train average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y_train,keep_prob: 1.0}) # model을 평가할 때는 전부 다 사용(keep_prob)
        # validation accuarcy
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # validation loss
        cur_val_loss = loss_op.eval({x: x_val, y: y_val,keep_prob: 1.0})
        
        print("epoch: "+str(epoch) + ", train acc: " + str(cur_train_acc) + ", val acc: " + str(cur_val_acc) )
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on "+str(epoch))
                    break
                else:
                    print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc

epoch: 0, train acc: 0.16508, val acc: 0.17
epoch: 1, train acc: 0.71804, val acc: 0.7426
epoch: 2, train acc: 0.85864, val acc: 0.8729
epoch: 3, train acc: 0.87476, val acc: 0.8848
epoch: 4, train acc: 0.88682, val acc: 0.8943
epoch: 5, train acc: 0.89334, val acc: 0.902
epoch: 6, train acc: 0.89786, val acc: 0.9045
epoch: 7, train acc: 0.90288, val acc: 0.9079
epoch: 8, train acc: 0.90562, val acc: 0.9086
epoch: 9, train acc: 0.90686, val acc: 0.9061
overfitting warning: 0
epoch: 10, train acc: 0.90616, val acc: 0.9078
epoch: 11, train acc: 0.90502, val acc: 0.9068
epoch: 12, train acc: 0.90418, val acc: 0.9048
epoch: 13, train acc: 0.89454, val acc: 0.8939
epoch: 14, train acc: 0.907, val acc: 0.9057
overfitting warning: 0
epoch: 15, train acc: 0.90422, val acc: 0.9002
epoch: 16, train acc: 0.91104, val acc: 0.9092
epoch: 17, train acc: 0.91262, val acc: 0.9087
overfitting warning: 0
epoch: 18, train acc: 0.92084, val acc: 0.916
epoch: 19, train acc: 0.90486, val acc: 0.9027
epoch: 

## **Test with the best epoch**

In [28]:
# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0})) # test 할 때도 모든 노드 사용(keep_prob)

INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.9628
